VGG19 Transfer Learning on Oxford 102 Flowers Dataset

Implements transfer learning with VGG19 and YOLOv5-CLS for flower classification.
- Random split: 50/25/25 (train/val/test)
- Run with different seeds: run_experiment(split_seed=1) and run_experiment(split_seed=2)
- Saves training curves and model checkpoints


In [1]:

import json
import math
import os
import random
from dataclasses import dataclass
from typing import Dict, List, Tuple
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import ConcatDataset, DataLoader, Dataset, Subset
from torchvision import datasets, models, transforms
import csv
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.io as sio
from torch.utils.data import Dataset
from PIL import Image


In [2]:

class Flowers102Local(Dataset):
    """
    Local Oxford 102 Flowers dataset:
    root/
      jpg/
      imagelabels.mat
      setid.mat
    """
    def __init__(self, root, transform=None):
        self.root = Path(root)
        self.img_dir = self.root / "jpg"
        self.transform = transform

        mat = sio.loadmat(self.root / "imagelabels.mat")
        labels = mat["labels"].squeeze().astype(int)  # 1..102
        self.labels = (labels - 1).tolist()           # 0..101

        self.image_paths = [
            self.img_dir / f"image_{i:05d}.jpg"
            for i in range(1, len(self.labels) + 1)
        ]

    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
            path = self.image_paths[idx]
            img = Image.open(path).convert("RGB")
            y = self.labels[idx]
            if self.transform:
                img = self.transform(img)
            return img, y, str(path)

In [3]:
# -----------------------------
# Dataset utilities
# -----------------------------
@dataclass
class SplitIndices:
    train: List[int]
    val: List[int]
    test: List[int]

class TransformOverrideDataset(Dataset):
    def __init__(self, base: Dataset, transform):
        self.base = base
        self.transform = transform

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        sample = self.base[idx]
        if len(sample) == 2:
            x, y = sample
            p = None
        else:
            x, y, p = sample

        if self.transform is not None:
            x = self.transform(x)

        return (x, y) if p is None else (x, y, p)



In [4]:
def load_flowers102_all(root: str) -> Dataset:
    return Flowers102Local(root=root, transform=None)


def make_random_split_indices(n: int, split_seed: int) -> SplitIndices:
    g = torch.Generator().manual_seed(split_seed)
    perm = torch.randperm(n, generator=g).tolist()

    n_train = int(0.50 * n)
    n_val = int(0.25 * n)
    n_test = n - n_train - n_val

    train_idx = perm[:n_train]
    val_idx = perm[n_train:n_train + n_val]
    test_idx = perm[n_train + n_val:n_train + n_val + n_test]

    return SplitIndices(train=train_idx, val=val_idx, test=test_idx)


def save_split_indices(path: str, split: SplitIndices) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump({"train": split.train, "val": split.val, "test": split.test}, f)


def load_split_indices(path: str) -> SplitIndices:
    with open(path, "r", encoding="utf-8") as f:
        obj = json.load(f)
    return SplitIndices(train=obj["train"], val=obj["val"], test=obj["test"])



In [5]:
def build_transforms(img_size: int = 224):
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]

    train_tf = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.02),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])

    eval_tf = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])

    return train_tf, eval_tf


In [8]:
def make_loaders(
    root: str,
    batch_size: int,
    num_workers: int,
    split_seed: int,
    split_cache_path: str,
    img_size: int = 224,
) -> Tuple[DataLoader, DataLoader, DataLoader, SplitIndices]:
    base_all = load_flowers102_all(root=root)
    n = len(base_all)

    if os.path.isfile(split_cache_path):
        split = load_split_indices(split_cache_path)
    else:
        split = make_random_split_indices(n=n, split_seed=split_seed)
        save_split_indices(split_cache_path, split)

    train_tf, eval_tf = build_transforms(img_size=img_size)

    train_ds = TransformOverrideDataset(Subset(base_all, split.train), transform=train_tf)
    val_ds   = TransformOverrideDataset(Subset(base_all, split.val), transform=eval_tf)
    test_ds  = TransformOverrideDataset(Subset(base_all, split.test), transform=eval_tf)

    # In Windows notebooks, num_workers>0 can cause issues. Use 0 unless you know it's stable.
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=True)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False,
                              num_workers=num_workers, pin_memory=True)
    test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False,
                              num_workers=num_workers, pin_memory=True)

    return train_loader, val_loader, test_loader, split


Models

vgg19_classifier

In [7]:
# -----------------------------
# Model
# -----------------------------
def build_vgg19_classifier(num_classes: int = 102, freeze_features: bool = True) -> nn.Module:
    model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)

    if freeze_features:
        for p in model.features.parameters():
            p.requires_grad = False

    in_features = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(in_features, num_classes)
    return model


yolov5_cls

In [21]:
def build_yolov5_cls_classifier(
    num_classes: int = 102,
    freeze_backbone: bool = True,
    weights: str = "yolov5s-cls.pt",
) -> nn.Module:
    """
    REAL YOLOv5 classifier using YOLOv5-CLS checkpoints (e.g., yolov5s-cls.pt).

    - No fallback. If YOLOv5-CLS can't load, it raises an error.
    - Replaces the final classifier layer to num_classes.
    """
    print(f"Loading REAL YOLOv5-CLS checkpoint: {weights}")

    # Load YOLOv5-CLS model from Ultralytics YOLOv5 repo via torch.hub
    yolo = torch.hub.load(
        "ultralytics/yolov5",
        "custom",
        path=weights,          # can be local path or model name if supported
        autoshape=False,       # raw torch model for training
        verbose=False,
    )

    model = getattr(yolo, "model", None)
    if model is None:
        raise RuntimeError("Unexpected YOLOv5 hub return: missing `.model` attribute.")

    # For YOLOv5-CLS, last layer is typically a Classify module with `.linear`
    head = model.model[-1]
    if not hasattr(head, "linear"):
        raise RuntimeError(
            "This does not look like a YOLOv5-CLS checkpoint (missing head.linear). "
            "You probably loaded a detection checkpoint (e.g., yolov5s.pt). "
            "Use yolov5s-cls.pt / yolov5m-cls.pt / yolov5l-cls.pt etc."
        )

    # Replace classifier
    in_features = head.linear.in_features
    head.linear = nn.Linear(in_features, num_classes)

    # Freeze backbone conditionally
    if freeze_backbone:
        for p in model.parameters():
            p.requires_grad = False
        for p in head.parameters():
            p.requires_grad = True

    return model


Resnet50

In [7]:
def build_resnet50_classifier(num_classes: int = 102, freeze_backbone: bool = True) -> nn.Module:
    """Build ResNet50 model for transfer learning."""
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

    if freeze_backbone:
        # Freeze all layers except the final classification layer
        for param in model.parameters():
            param.requires_grad = False
    
    # Replace the final fully connected layer
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    
    # Always unfreeze the final layer
    for param in model.fc.parameters():
        param.requires_grad = True
        
    return model

In [9]:
@torch.no_grad()
def evaluate(model, loader, device, criterion):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    for batch in loader:
        if len(batch) == 2:
            x, y = batch
        else:
            x, y, _paths = batch

        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        logits = model(x)
        loss = criterion(logits, y)

        total_loss += loss.item() * x.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == y).sum().item()
        total += x.size(0)

    return total_loss / max(1, total), correct / max(1, total)


In [10]:
def train_one_epoch(model, loader, device, criterion, optimizer):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for batch in loader:
        if len(batch) == 2:
            x, y = batch
        else:
            x, y, _paths = batch  # ignore paths during training

        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == y).sum().item()
        total += x.size(0)

    return total_loss / max(1, total), correct / max(1, total)


In [11]:
def save_curves(out_dir: str, history: Dict[str, List[float]]) -> None:
    os.makedirs(out_dir, exist_ok=True)

    plt.figure()
    plt.plot(history["train_acc"], label="train")
    plt.plot(history["val_acc"], label="val")
    plt.plot(history["test_acc"], label="test")
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "accuracy_vgg19.png"), dpi=160)
    plt.close()

    plt.figure()
    plt.plot(history["train_loss"], label="train")
    plt.plot(history["val_loss"], label="val")
    plt.plot(history["test_loss"], label="test")
    plt.xlabel("epoch")
    plt.ylabel("cross_entropy")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "loss_vgg19.png"), dpi=160)
    plt.close()

    with open(os.path.join(out_dir, "history_vgg19.json"), "w", encoding="utf-8") as f:
        json.dump(history, f, indent=2)


In [12]:

def save_checkpoint(path: str, model: nn.Module, optimizer: optim.Optimizer, epoch: int, best_val_acc: float) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(
        {"epoch": epoch, "model_state": model.state_dict(),
         "optimizer_state": optimizer.state_dict(), "best_val_acc": best_val_acc},
        path,
    )



In [13]:

@torch.no_grad()
def predict_proba(model: nn.Module, images: torch.Tensor, device: torch.device) -> torch.Tensor:
    model.eval()
    images = images.to(device)
    logits = model(images)
    return torch.softmax(logits, dim=1).cpu()



In [14]:

@torch.no_grad()
def export_probs_csv(model, loader, device, out_csv: str):
    model.eval()
    header = ["path", "true_label"] + [f"p_{i:03d}" for i in range(102)]

    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(header)

        for batch in loader:
            if len(batch) == 2:
                images, labels = batch
                paths = ["" for _ in range(images.size(0))]
            else:
                images, labels, paths = batch

            images = images.to(device, non_blocking=True)
            probs = torch.softmax(model(images), dim=1).cpu()  # (B,102)

            for i in range(probs.size(0)):
                row = [paths[i], int(labels[i])] + probs[i].tolist()
                w.writerow(row)

    print("Saved:", out_csv)


In [23]:
def run_experiment(
    model_type: str = "vgg19",  # "vgg19", "resnet50", or "yolov5"
    data_root: str = "./data",
    out_dir: str = "./results/vgg19_seed1",
    epochs: int = 35,
    batch_size: int = 32,
    lr: float = 1e-4,
    weight_decay: float = 0.0,
    num_workers: int = 0,
    img_size: int = 224,
    split_seed: int = 1,
    freeze_backbone: bool = True,
    early_stop_patience: int = 7,
    device: str = "cuda",
):
    """
    Main training function for flower classification.
    
    Args:
        model_type: "vgg19", "resnet50", or "yolov5"
        data_root: Path to dataset root directory
        out_dir: Output directory for results
        epochs: Maximum number of training epochs
        batch_size: Batch size for training
        lr: Learning rate (will be adjusted based on model_type)
        weight_decay: Weight decay for optimizer
        num_workers: Number of data loader workers
        img_size: Input image size
        split_seed: Random seed for data split
        freeze_backbone: Whether to freeze pre-trained backbone
        early_stop_patience: Early stopping patience
        device: "cuda" or "cpu"
    
    Returns:
        history: Dictionary with training history
        ckpt_path: Path to best model checkpoint
    """
    
    # Set device
    if device == "cuda" and not torch.cuda.is_available():
        print("CUDA requested but not available. Falling back to CPU.")
        device = "cpu"
    device_t = torch.device(device)
    
    # Create output directory
    os.makedirs(out_dir, exist_ok=True)
    
    # Set random seeds for reproducibility
    set_seed(split_seed)
    
    # Split cache path
    split_cache_path = os.path.join(out_dir, f"split_indices_seed_{split_seed}.json")
    
    # Create data loaders
    train_loader, val_loader, test_loader, _split = make_loaders(
        root=data_root,
        batch_size=batch_size,
        num_workers=num_workers,
        split_seed=split_seed,
        split_cache_path=split_cache_path,
        img_size=img_size,
    )
    
    # Build model based on type
    print(f"\n{'='*60}")
    print(f"Training {model_type.upper()} - Split Seed: {split_seed}")
    print(f"{'='*60}")
    
    # Initialize variables
    model = None
    unfrozen_params = None
    model_name = model_type.lower()  # Define model_name here
    
    if model_name == "vgg19":
        # VGG19 model
        model = build_vgg19_classifier(num_classes=102, freeze_features=freeze_backbone)
        if lr == 1e-4:  # Default value
            lr = 1e-4  # Good starting point for VGG19
        # Get unfrozen parameters (only the last classifier layer if frozen)
        if freeze_backbone:
            unfrozen_params = [p for p in model.classifier[-1].parameters()]
        else:
            unfrozen_params = model.parameters()
            
    elif model_name == "resnet50":
        # ResNet50 model
        model = build_resnet50_classifier(num_classes=102, freeze_backbone=freeze_backbone)
        if lr == 1e-4:  # Default value
            lr = 1e-4  # Good starting point for ResNet50
        # Get unfrozen parameters
        if freeze_backbone:
            unfrozen_params = [p for p in model.fc.parameters()]
        else:
            unfrozen_params = model.parameters()
            
    elif model_name in ["yolov5_cls", "yolov5-cls", "yolov5"]:
        # YOLOv5-based model
        model = build_yolov5_cls_classifier(num_classes=102, freeze_backbone=freeze_backbone)
        model_name = "yolov5"  # Standardize name
        if lr == 1e-4:  # Default value
            lr = 5e-5  # Lower learning rate for YOLOv5
        # Get unfrozen parameters
        if freeze_backbone:
            head = model.model[-1]  # Classify module
            unfrozen_params = list(head.parameters())
        else:
            unfrozen_params = model.parameters()
    else:
        raise ValueError(f"Unknown model type: {model_type}. Choose from 'vgg19', 'resnet50', or 'yolov5'.")
    
    # Safety check
    if model is None:
        raise RuntimeError("Model was not built properly.")
    if unfrozen_params is None:
        raise RuntimeError("Unfrozen parameters were not set properly.")
    
    # Move model to device
    model = model.to(device_t)
    
    # Print model summary
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in unfrozen_params)
    print(f"Model: {model_name}")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Frozen parameters: {total_params - trainable_params:,}")
    print(f"Learning rate: {lr}")
    print(f"Batch size: {batch_size}")
    print(f"Epochs: {epochs}")
    
    # Define optimizer - only optimize trainable parameters
    optimizer = optim.Adam(unfrozen_params, lr=lr, weight_decay=weight_decay)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3)
    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Initialize history tracking
    history = {
        "train_loss": [], "val_loss": [], "test_loss": [],
        "train_acc": [], "val_acc": [], "test_acc": [],
        "learning_rate": []
    }
    
    # Early stopping variables
    best_val_acc = -1.0
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_epoch = 0
    
    # Checkpoint path
    ckpt_path = os.path.join(out_dir, f"best_{model_name}.pt")
    
    # Training loop
    print(f"\nStarting training for {epochs} epochs...")
    print("-" * 80)
    
    for epoch in range(1, epochs + 1):
        # Training phase
        tr_loss, tr_acc = train_one_epoch(model, train_loader, device_t, criterion, optimizer)
        
        # Validation phase
        va_loss, va_acc = evaluate(model, val_loader, device_t, criterion)
        
        # Test phase (for monitoring, not for early stopping)
        te_loss, te_acc = evaluate(model, test_loader, device_t, criterion)
        
        # Update learning rate scheduler
        scheduler.step(va_loss)
        
        # Record history
        history["train_loss"].append(tr_loss)
        history["val_loss"].append(va_loss)
        history["test_loss"].append(te_loss)
        history["train_acc"].append(tr_acc)
        history["val_acc"].append(va_acc)
        history["test_acc"].append(te_acc)
        history["learning_rate"].append(optimizer.param_groups[0]['lr'])
        
        # Print progress
        print(
            f"Epoch {epoch:03d}/{epochs} | "
            f"Train: loss {tr_loss:.4f} acc {tr_acc:.4f} | "
            f"Val: loss {va_loss:.4f} acc {va_acc:.4f} | "
            f"Test: loss {te_loss:.4f} acc {te_acc:.4f} | "
            f"LR: {history['learning_rate'][-1]:.2e}"
        )
        
        # Check for improvement in validation accuracy
        if va_acc > best_val_acc:
            best_val_acc = va_acc
            best_val_loss = va_loss
            best_epoch = epoch
            epochs_no_improve = 0
            
            # Save best model checkpoint
            save_checkpoint(ckpt_path, model, optimizer, epoch, best_val_acc)
            print(f"✓ New best model saved! Val Acc: {best_val_acc:.4f}")
            
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= early_stop_patience:
                print(f"\nEarly stopping triggered at epoch {epoch}. "
                      f"No improvement for {early_stop_patience} epochs.")
                break
        
        # Additional: save checkpoint every 5 epochs
        if epoch % 5 == 0 and epoch != best_epoch:
            intermediate_ckpt = os.path.join(out_dir, f"{model_name}_epoch_{epoch}.pt")
            save_checkpoint(intermediate_ckpt, model, optimizer, epoch, va_acc)
    
    # Final evaluation on test set with best model
    print("\n" + "="*60)
    print("FINAL EVALUATION")
    print("="*60)
    
    # Load best model
    checkpoint = torch.load(ckpt_path, map_location=device_t)
    model.load_state_dict(checkpoint['model_state'])
    
    # Evaluate on test set
    final_test_loss, final_test_acc = evaluate(model, test_loader, device_t, criterion)
    
    # Save training curves
    save_curves(out_dir, history, model_name=model_name)
    
    # Export probabilities
    export_probs_csv(
        model=model,
        loader=test_loader,
        device=device_t,
        out_csv=os.path.join(out_dir, f"test_probs_{model_name}_seed{split_seed}.csv"),
    )

    # Save full history
    history_path = os.path.join(out_dir, f"history_{model_name}.json")
    with open(history_path, "w", encoding="utf-8") as f:
        json.dump({k: [float(v) for v in vals] for k, vals in history.items()}, f, indent=2)
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"{model_name.upper()} TRAINING SUMMARY")
    print(f"{'='*60}")
    print(f"Data split seed: {split_seed}")
    print(f"Best epoch: {best_epoch}")
    print(f"Best validation accuracy: {best_val_acc:.4f}")
    print(f"Best validation loss: {best_val_loss:.4f}")
    print(f"Final test accuracy: {final_test_acc:.4f}")
    print(f"Final test loss: {final_test_loss:.4f}")
    print(f"Total epochs trained: {len(history['train_acc'])}")
    
    print(f"\nSaved files:")
    print(f"- Checkpoint: {ckpt_path}")
    print(f"- Accuracy curve: {os.path.join(out_dir, f'training_curves_{model_name}.png')}")
    print(f"- Loss curve: {os.path.join(out_dir, f'training_curves_{model_name}.png')}")
    print(f"- History: {history_path}")
    print(f"- Split indices: {split_cache_path}")
    print(f"- Test probabilities: {os.path.join(out_dir, f'test_probs_{model_name}_seed{split_seed}.csv')}")
    
    # Add final test metrics to history
    history["final_test_acc"] = final_test_acc
    history["final_test_loss"] = final_test_loss
    history["best_val_acc"] = best_val_acc
    history["best_epoch"] = best_epoch
    history["model_type"] = model_name
    history["split_seed"] = split_seed

    return history, ckpt_path

In [16]:


# Modified save_curves to support both models
def save_curves(out_dir: str, history: Dict[str, List[float]], model_name: str = "model") -> None:
    """
    Save accuracy and loss curves for the model.
    """
    os.makedirs(out_dir, exist_ok=True)
    
    # Accuracy plot
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history["train_acc"], label="Train", linewidth=2)
    plt.plot(history["val_acc"], label="Validation", linewidth=2)
    plt.plot(history["test_acc"], label="Test", linewidth=2)
    plt.axhline(y=0.70, color='gray', linestyle='--', alpha=0.7, label='70% Requirement')
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"{model_name.upper()} - Accuracy")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history["train_loss"], label="Train", linewidth=2)
    plt.plot(history["val_loss"], label="Validation", linewidth=2)
    plt.plot(history["test_loss"], label="Test", linewidth=2)
    plt.xlabel("Epoch")
    plt.ylabel("Cross-Entropy Loss")
    plt.title(f"{model_name.upper()} - Loss")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"training_curves_{model_name}.png"), dpi=150, bbox_inches='tight')
    plt.close()
    
    # Learning rate plot
    if "learning_rate" in history:
        plt.figure(figsize=(6, 4))
        plt.plot(history["learning_rate"], linewidth=2)
        plt.xlabel("Epoch")
        plt.ylabel("Learning Rate")
        plt.title(f"{model_name.upper()} - Learning Rate Schedule")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, f"learning_rate_{model_name}.png"), dpi=150, bbox_inches='tight')
        plt.close()

In [17]:
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    # Determinism (can reduce speed; )
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [34]:
h1, ckpt1 = run_experiment(
    data_root=r"C:\Users\hp\Downloads\102flowers",
    split_seed=1,
    out_dir="./results/vgg19_seed1"
)



Training VGG19 - Split Seed: 1
Model: vgg19
Total parameters: 139,988,134
Trainable parameters: 417,894
Frozen parameters: 139,570,240
Learning rate: 0.0001

Starting training for 35 epochs...
--------------------------------------------------------------------------------
Epoch 001/35 | Train: loss 4.0681 acc 0.1136 | Val: loss 3.4840 acc 0.3214 | Test: loss 3.5004 acc 0.3071 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.3214
Epoch 002/35 | Train: loss 3.1044 acc 0.3356 | Val: loss 2.8805 acc 0.4455 | Test: loss 2.9096 acc 0.4224 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.4455
Epoch 003/35 | Train: loss 2.5820 acc 0.4673 | Val: loss 2.4969 acc 0.5266 | Test: loss 2.5175 acc 0.5122 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.5266
Epoch 004/35 | Train: loss 2.2639 acc 0.5305 | Val: loss 2.2330 acc 0.5843 | Test: loss 2.2610 acc 0.5771 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.5843
Epoch 005/35 | Train: loss 2.0190 acc 0.5850 | Val: loss 2.0416 acc 0.6170 | Test

In [19]:
h2, ckpt2 = run_experiment(
    data_root=r"C:\Users\hp\Downloads\102flowers",
    split_seed=2,
    out_dir="./results/vgg19_seed2"
)


Training VGG19 - Split Seed: 2
Model: vgg19
Total parameters: 139,988,134
Trainable parameters: 417,894
Frozen parameters: 139,570,240
Learning rate: 0.0001

Starting training for 35 epochs...
--------------------------------------------------------------------------------
Epoch 001/35 | Train: loss 4.0723 acc 0.1197 | Val: loss 3.4878 acc 0.2995 | Test: loss 3.4551 acc 0.3154 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.2995
Epoch 002/35 | Train: loss 3.1211 acc 0.3339 | Val: loss 2.8876 acc 0.4372 | Test: loss 2.8433 acc 0.4619 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.4372
Epoch 003/35 | Train: loss 2.6091 acc 0.4511 | Val: loss 2.5152 acc 0.5242 | Test: loss 2.4717 acc 0.5605 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.5242
Epoch 004/35 | Train: loss 2.2734 acc 0.5286 | Val: loss 2.2528 acc 0.5760 | Test: loss 2.2079 acc 0.6094 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.5760
Epoch 005/35 | Train: loss 2.0338 acc 0.5794 | Val: loss 2.0709 acc 0.5984 | Test

Yolov5 fixed

In [26]:
print("\n=== REAL YOLOv5-CLS Experiments ===")

h1_yolo_real, ckpt1_yolo_real = run_experiment(
    model_type="yolov5_cls",
    data_root=r"C:\Users\hp\Downloads\102flowers",
    split_seed=1,
    out_dir="./results/yolov5_cls_seed1",
    epochs=30,
    lr=5e-5,
    freeze_backbone=True,
)



=== REAL YOLOv5-CLS Experiments ===

Training YOLOV5_CLS - Split Seed: 1
Loading REAL YOLOv5-CLS checkpoint: yolov5s-cls.pt


YOLOv5  2026-1-14 Python-3.13.5 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)



Model: yolov5
Total parameters: 4,303,142
Trainable parameters: 788,582
Frozen parameters: 3,514,560
Learning rate: 5e-05

Starting training for 30 epochs...
--------------------------------------------------------------------------------
Epoch 001/30 | Train: loss 4.2718 acc 0.1063 | Val: loss 3.8966 acc 0.2428 | Test: loss 3.9262 acc 0.2139 | LR: 5.00e-05
✓ New best model saved! Val Acc: 0.2428
Epoch 002/30 | Train: loss 3.5577 acc 0.3136 | Val: loss 3.2747 acc 0.3884 | Test: loss 3.3098 acc 0.3711 | LR: 5.00e-05
✓ New best model saved! Val Acc: 0.3884
Epoch 003/30 | Train: loss 2.9450 acc 0.4934 | Val: loss 2.7561 acc 0.5032 | Test: loss 2.7872 acc 0.5015 | LR: 5.00e-05
✓ New best model saved! Val Acc: 0.5032
Epoch 004/30 | Train: loss 2.4163 acc 0.6087 | Val: loss 2.3015 acc 0.6185 | Test: loss 2.3276 acc 0.6265 | LR: 5.00e-05
✓ New best model saved! Val Acc: 0.6185
Epoch 005/30 | Train: loss 1.9961 acc 0.7171 | Val: loss 1.9385 acc 0.6790 | Test: loss 1.9541 acc 0.6909 | LR: 5.00e

In [27]:

h2_yolo_real, ckpt2_yolo_real = run_experiment(
    model_type="yolov5_cls",
    data_root=r"C:\Users\hp\Downloads\102flowers",
    split_seed=2,
    out_dir="./results/yolov5_cls_seed2",
    epochs=30,
    lr=5e-5,
    freeze_backbone=True,
)


YOLOv5  2026-1-14 Python-3.13.5 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)




Training YOLOV5_CLS - Split Seed: 2
Loading REAL YOLOv5-CLS checkpoint: yolov5s-cls.pt
Model: yolov5
Total parameters: 4,303,142
Trainable parameters: 788,582
Frozen parameters: 3,514,560
Learning rate: 5e-05

Starting training for 30 epochs...
--------------------------------------------------------------------------------
Epoch 001/30 | Train: loss 4.2770 acc 0.1295 | Val: loss 3.9111 acc 0.2296 | Test: loss 3.8800 acc 0.2515 | LR: 5.00e-05
✓ New best model saved! Val Acc: 0.2296
Epoch 002/30 | Train: loss 3.5514 acc 0.3129 | Val: loss 3.3129 acc 0.3810 | Test: loss 3.2716 acc 0.4111 | LR: 5.00e-05
✓ New best model saved! Val Acc: 0.3810
Epoch 003/30 | Train: loss 2.9393 acc 0.5000 | Val: loss 2.7868 acc 0.5056 | Test: loss 2.7405 acc 0.5215 | LR: 5.00e-05
✓ New best model saved! Val Acc: 0.5056
Epoch 004/30 | Train: loss 2.4117 acc 0.6241 | Val: loss 2.3262 acc 0.6014 | Test: loss 2.2819 acc 0.6147 | LR: 5.00e-05
✓ New best model saved! Val Acc: 0.6014
Epoch 005/30 | Train: loss 2.

In [25]:
# Initialize results dictionary
all_results = {}

# Run ResNet50 experiments
print("\n" + "="*80)
print("RESNET50 EXPERIMENTS")
print("="*80)


RESNET50 EXPERIMENTS


In [ ]:
h1_resnet, ckpt1_resnet = run_experiment(
        model_type="resnet50",
        data_root="C:\\Users\\hp\\Downloads\\102flowers",
        split_seed=1,
        out_dir="./results/resnet50_seed1",
        epochs=35,
        lr=1e-4,
        freeze_backbone=True,
    )



Training RESNET50 - Split Seed: 1
Model: resnet50
Total parameters: 23,717,030
Trainable parameters: 208,998
Frozen parameters: 23,508,032
Learning rate: 0.0001
Batch size: 32
Epochs: 35

Starting training for 35 epochs...
--------------------------------------------------------------------------------
Epoch 001/35 | Train: loss 4.2580 acc 0.0984 | Val: loss 3.8844 acc 0.2086 | Test: loss 3.9039 acc 0.1851 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.2086
Epoch 002/35 | Train: loss 3.6169 acc 0.3000 | Val: loss 3.3204 acc 0.4045 | Test: loss 3.3413 acc 0.3994 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.4045
Epoch 003/35 | Train: loss 3.0968 acc 0.4778 | Val: loss 2.8569 acc 0.5369 | Test: loss 2.8847 acc 0.5371 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.5369
Epoch 004/35 | Train: loss 2.6551 acc 0.5989 | Val: loss 2.4665 acc 0.6019 | Test: loss 2.4999 acc 0.6133 | LR: 1.00e-04
✓ New best model saved! Val Acc: 0.6019
Epoch 005/35 | Train: loss 2.2971 acc 0.6776 | Val:

NameError: name 'all_results' is not defined

In [26]:
all_results["resnet50_seed1"] = h1_resnet

In [ ]:
h2_resnet, ckpt2_resnet = run_experiment(
        model_type="resnet50",
        data_root="C:\\Users\\hp\\Downloads\\102flowers",
        split_seed=2,
        out_dir="./results/resnet50_seed2",
        epochs=35,
        lr=1e-4,
        freeze_backbone=True,
    )
all_results["resnet50_seed2"] = h2_resnet
    


Training RESNET50 - Split Seed: 2
Model: resnet50
Total parameters: 23,717,030
Trainable parameters: 208,998
Frozen parameters: 23,508,032
Learning rate: 0.0001
Batch size: 32
Epochs: 35

Starting training for 35 epochs...
--------------------------------------------------------------------------------


In [ ]:
# Run all three models comparison
all_results = {}
all_results["vgg19_seed1"] = h1_vgg  # from your previous run
all_results["vgg19_seed2"] = h2_vgg  # from your previous run
all_results["resnet50_seed1"] = h1_resnet
all_results["resnet50_seed2"] = h2_resnet
all_results["yolov5_seed1"] = h1_yolo  # from your previous run
all_results["yolov5_seed2"] = h2_yolo  # from your previous run

# Generate comparison report
generate_comparison_report(all_results)
This will give you a complete comparison bet